In [ ]:
!pip install Sastrawi

     |████████████████████████████████| 209 kB 5.3 MB/s 


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import keras
from keras import optimizers
from keras import backend as K
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from tensorflow.keras import layers, initializers
from tensorflow.keras.layers import Concatenate, Input, Embedding, LSTM, Attention, Dense, Dropout, Bidirectional, LSTM, Flatten, Conv1D, GlobalMaxPool1D, MaxPool1D, Reshape, GlobalAveragePooling1D, UpSampling1D, Layer

from subprocess import check_output

import tensorflow
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
import datetime
import sys
import os
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer 
import os, re, csv, math, codecs

sns.set_style("whitegrid")
np.random.seed(0)
nltk.download('punkt')

tokenizer = RegexpTokenizer(r'\w+')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# for mounting to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# read data train and test
train_df = pd.read_csv('/content/drive/fasttext/train9000.csv', sep=',', header=0)
test_df = pd.read_csv('/content/drive/fasttext/test900.csv', sep=',', header=0)

print("num train: ", train_df.shape[0])
print("num test: ", test_df.shape[0])

num train:  9737
num test:  900


In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

factoryStopword = StopWordRemoverFactory()
stopword = factoryStopword.create_stop_word_remover()
stop_factory = StopWordRemoverFactory().get_stop_words()

#removes html
def strip_html(text):
  soup = BeautifulSoup(text, "html.parser")
  return soup.get_text()
  
#remove between square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#eliminates contractions
def replace_contractions(text):
  """Replace contactions in string of text"""
  return contractions.fix(text)

#denoise text
def denoise_text(text):
  text = strip_html(text)
  text = remove_between_square_brackets(text)
  text = replace_contractions(text)
  return text

#cleaning dataset
def cleaning(str):
    #remove non-ascii
    str = unicodedata.normalize('NFKD', str).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    str = re.sub("b'|b\"",'',str)
    #remove username inside brackets
    str = re.sub('SENSITIVE-NO',' ',str) 
    #Remove additional white spaces
    str = re.sub('[\s]+', ' ', str)
    #remove URLs
    str = re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', str)
    #remove punctuations
    str = re.sub(r'[^\w]|_',' ',str)
    #remove digit from string
    str = re.sub("\S*\d\S*", "", str).strip()
    #remove digit or numbers
    str = re.sub(r"\b\d+\b", " ", str)
    #to lowercase
    str = str.lower()
    return str

#remove twitter symbols
def remove_symbols(str):
  #remove RT
  str = re.sub ('RT', '', str)
  #remove @username
  str = re.sub('@[^\s]+', '', str)
  return str

#normalization of words
normalisasi = pd.read_csv('kamus.csv', encoding = 'latin-1')
normalisasi_map = dict(zip(normalisasi['original'], normalisasi['replacement']))
def normalizing(text):
  return ' '.join([normalisasi_map[word] if word in normalisasi_map else word for word in text.split(' ')])


def get_stopwords():
  docs = pd.read_csv('/content/stopwords-id.txt', header = None, names= ['stopword'])
  docs = docs['stopword'].tolist()
  return docs

stop_words = get_stopwords()

# Merge stopword
data = set(stop_factory + stop_words)

#remove stopword
def removeStopword(str):
    word_tokens = word_tokenize(str)
    filtered_sentence = [w for w in word_tokens if not w in data]
    return ' '.join(filtered_sentence) 

def processing(str):
  str = remove_symbols(str)
  str = denoise_text(str)
  str = normalizing(str)
  str = cleaning(str)
  str = stemming(str)
  return str

In [ ]:
# preprocessing
train_df['sentiment'] = train_df['sentiment'].apply(processing)
test_df['sentiment'] = test_df['sentiment'].apply(processing)

# merge
# merge columns
train_df['sentiment'] = train_df[train_df.columns[:2]].apply(
    lambda x: ' '.join(x.dropna().astype(str)), axis=1)

test_df['sentiment'] = test_df[test_df.columns[:2]].apply(
    lambda x: ' '.join(x.dropna().astype(str)), axis=1)

In [ ]:
train_df.to_csv('balance-train-final.csv', index = False)
test_df.to_csv('balance-test-final.csv', index = False)

In [ ]:
raw_docs_train = train_df['sentiment'].tolist()
raw_docs_test = test_df['sentiment'].tolist() 
num_classes = 3

In [ ]:
print("pre-processing train data...")
processed_docs_train = []
for doc in tqdm(raw_docs_train):
    doc = re.sub(r'\b(halaman|halaman all|all)\b', '', doc)
    doc = re.sub(r'\b(com|co id|net|id|news)\b', '', doc)
    doc = re.sub(r'\b(news)\b', '', doc)
    doc = re.sub(r'\b(tribun|tribunnews|fimela)', '', doc)
    tokens = tokenizer.tokenize(doc)
    #filtered = [word for word in tokens if word not in data]
    processed_docs_train.append(doc)

print(raw_docs_train)
print(processed_docs_train)

np.savetxt('trainheadline.txt', processed_docs_train, delimiter=' ', fmt="%s")
! wc trainheadline.txt
! head -n 6000 trainheadline.txt > headlineImbalance.train

pre-processing train data...


100%|██████████| 6000/6000 [00:00<00:00, 37056.63it/s]

['__label__POSITIVE buat para tebar benci beefikirlah yang positif lihat kenyataanya dong', '__label__POSITIVE aku ada solar ini buat ke pulau natuna ikut bela indonesia', '__label__POSITIVE kami rakyat indonesia ingin ibu kembali jadi menteri lagi', '__label__POSITIVE bukti iya kpk jilid ini lebih baik dari yang belum belum lama lantik sudah heboh dengan dua ott besar bukti dewan awas dan komisioner bisa koordinasi dengan baik moga ke depan kpk lebih baik', '__label__POSITIVE mau pesan tiket dan muncul informasi banjir moga cepat surut', '__label__POSITIVE kami anakdesa juga turut berangkat kena kapal menujun natuna untuk aman para nelayan airmin siap tunggu perintah bu natunamilikindonesia jokowikawalnatuna', '__label__POSITIVE bagi sertifikat di natuna tolong bagi sedikit perhati kepada pak robert yang beli tanah lelang negara tapi enggak ada tanah negara tipu', '__label__POSITIVE indonesia kalau lagi musim hujan semua diskusi cara cegah banjir nanti sudah musim kemarau semua diskus

  6000 116058 794511 trainheadline.txt


In [ ]:
processed_docs_test = []
for doc in tqdm(raw_docs_test):
    doc = re.sub(r'\b(halaman|halaman all|all)\b', '', doc)
    doc = re.sub(r'\b(com|co id|net|id|news)\b', '', doc)
    doc = re.sub(r'\b(news)\b', '', doc)
    doc = re.sub(r'\b(tribun|tribunnews|fimela)', '', doc)
    tokens = tokenizer.tokenize(doc)
    #filtered = [word for word in tokens if word not in data]
    processed_docs_test.append(doc)

print(raw_docs_test)
print(processed_docs_test)

np.savetxt('testheadline.txt', processed_docs_test, delimiter=' ', fmt="%s")
! wc testheadline.txt
! head -n 1875 testheadline.txt > headline.test

100%|██████████| 1875/1875 [00:00<00:00, 36277.05it/s]

['__label__POSITIVE pade repot amat iya biarin saja kenapa kpk kerja dengan cara kalau tidak ada kpk siapa yang mau berantas korupsi polisi mau bikin cegah yang seperti apa hampir tiap instansi larang kok korupsi nyata korupsi makin gila', '__label__POSITIVE natuna milik indonesia kita usir dan halau kapal kapal asing dengan segala mampu yang kiya milik kita akan pertqhankan daulat negara satu republik indonesia', '__label__POSITIVE bumi kita makin tua dan banyak tangan tangan nakal yang rusak tdk heran kalau bumi kita mulai kesal dan tumpah lewat banjir dan bagai apakah kita harus diam saja tentu tidak kita harus jaga bumi kita agar hijau hutan lestari laut bukan cuma sejarah untuk anak cucu kita', '__label__POSITIVE mantap mantap mantap hidup bendera merah putih hidup pak presiden joko widodo', '__label__POSITIVE moga beri harap baru dan lebih semangat lagi untuk kerja dan karya bagi warga yang telah dapat sertifikat tanah selalu sehat bapak menteri', '__label__POSITIVE jangan karena

In [ ]:
# downloading fasttext facebook
!wget https://github.com/facebookresearch/fastText/archive/v0.9.2.zip
!unzip v0.9.2.zip
%cd fastText-0.9.2
!make
!pip install .

--2022-01-05 07:02:53--  https://github.com/facebookresearch/fastText/archive/v0.9.2.zip
Resolving github.com (github.com)... 13.114.40.48
Connecting to github.com (github.com)|13.114.40.48|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/facebookresearch/fastText/zip/v0.9.2 [following]
--2022-01-05 07:02:53--  https://codeload.github.com/facebookresearch/fastText/zip/v0.9.2
Resolving codeload.github.com (codeload.github.com)... 13.112.159.149
Connecting to codeload.github.com (codeload.github.com)|13.112.159.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘v0.9.2.zip’

v0.9.2.zip              [  <=>               ]   4.17M  11.4MB/s    in 0.4s    

2022-01-05 07:02:54 (11.4 MB/s) - ‘v0.9.2.zip’ saved [4369852]

Archive:  v0.9.2.zip
5b5943c118b0ec5fb9cd8d20587de2b2d3966dfe
   creating: fastText-0.9.2/
   creating: fastText-0.9.2/.circleci/
  inflating: fastTe

In [ ]:
import fasttext

training_data_path ='/content/headlineImbalance.train' 
validation_data_path ='/content/headline.test'
model_path ='/content/'
model_name="model-headline"
vec_docs = "/content/drive/fasttext/fasttext-indo4b.vec" # using indo4b vector

def train():
    print('Training start')
    try:
        hyper_params = {"lr": 0.008,
                        "epoch": 10,
                        "wordNgrams": 4,
                        "dim": 300,
                        "pretrainedVectors": vec_docs
                        }     
                               
        print(str(datetime.datetime.now()) + ' START=>' + str(hyper_params) )

        # Train the model.
        global model
        model = fasttext.train_supervised(input=training_data_path, **hyper_params)
        print("Model trained with the hyperparameter \n {}".format(hyper_params))

        # CHECK PERFORMANCE
        print(str(datetime.datetime.now()) + 'Training complete.' + str(hyper_params) )
        
        result = model.test(training_data_path)
        validation = model.test(validation_data_path)
        
        # DISPLAY ACCURACY OF TRAINED MODEL
        text_line = str(hyper_params) + ",accuracy:" + str(result[1])  + ",validation:" + str(validation[1]) + '\n' 
        print(text_line)
        
        #quantize a model to reduce the memory usage
        model.quantize(input=training_data_path, qnorm=True, retrain=True, cutoff=100000)
        print("Model is quantized!!")
        model.save_model(os.path.join(model_path,model_name + ".ftz"))                
    
        #TESTING        
        print(model.predict(['dasar kadrun pemakar kurang ajar'],k=3)) #neg
        print(model.predict(['terima kasih beb'],k=1)) #pos
        print(model.predict(['kenal sih anu ga sih'], k=1)) #neu
        
        
    except Exception as e:
        print('Exception during training: ' + str(e) )


# Train your model.
train()

Training start
2021-10-11 05:57:39.775096 START=>{'lr': 0.008, 'epoch': 10, 'wordNgrams': 4, 'dim': 300, 'pretrainedVectors': '/content/drive/MyDrive/College/Careers/Accepted Job/04 Solusi247 - Data Scientist/Jobs/fasttext/fasttext-indo4b.vec'}


In [ ]:
 print(model.predict(['bajingan emang dsr buzzer'],k=1)) #neg
 print(model.predict(['muhasabah diri lah harusnya'],k=1)) #pos
 print(model.predict(['perlu ditera[kan saat sedang darurat'], k=1)) #neu

([['__label__NEGATIVE']], [array([0.5491343], dtype=float32)])
([['__label__POSITIVE']], [array([0.72881335], dtype=float32)])
([['__label__POSITIVE']], [array([0.72881335], dtype=float32)])


In [ ]:
# y_true
fmov=open('/content/testheadline.txt','r')
sentence_list=[]
label_list=[]
for f in fmov.readlines():
    lineinfo=f.split(' ')
    sentenceline=".".join(lineinfo[0:-1])
    sentence_list.append(sentenceline)
    label_list.append(str(lineinfo[0]).replace('\n',''))
    y_true = label_list

In [ ]:
# y_pred
y_pred = model.predict(processed_docs_test, k=1)
y_pred2 = list(y_pred)
y_pred2 = y_pred2.pop(0)
out = np.concatenate(y_pred2).ravel().tolist()
y_pred = out

In [ ]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
print("confusion matrix: \n", confusion_matrix(y_true, y_pred))

# Accuracy
print("accuracy: ", accuracy_score(y_true, y_pred))

# Recall
from sklearn.metrics import recall_score
print("recall: \n", recall_score(y_true, y_pred, average=None))

# Precision
from sklearn.metrics import precision_score
print("precision: \n", precision_score(y_true, y_pred, average=None))

confusion matrix: 
 [[256 233 136]
 [ 88 233 304]
 [109 111 405]]
accuracy:  0.4768
recall: 
 [0.4096 0.3728 0.648 ]
precision: 
 [0.56512141 0.40381282 0.47928994]
